In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import random
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
import time
import csv


# Парсим ссылки

In [3]:
# for i in range(318,1086):
#     userAgent = ua.random
#     print(f"Текущий userAgent: {userAgent}, i = {i}")
    
#     options = webdriver.ChromeOptions()
#     options.add_argument(f"user-agent={userAgent}")
    
#     # Инициализация драйвера с новыми опциями
#     driver = webdriver.Chrome(options=options)
    
#     driver.get(f"https://realt.by/belarus/sale/flats/?page={i}")
#     elements = driver.find_elements(By.XPATH, value='//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div/div[1]/div[2]/div[1]/div/div')
#     for e in elements[0].find_elements(By.XPATH, '//a[contains(@aria-label, "' + 'Ссылка на объект' + '")]'):
#         href = e.get_attribute('href')
#         with open('link.txt', 'a') as file:
#             print(href, file=file)



In [4]:
# # Создание объекта UserAgent
# ua = UserAgent()

# for i in range(148, 1086):
#     # Генерация нового userAgent
#     userAgent = ua.random
#     print(f"Текущий userAgent: {userAgent}")
    
#     options = webdriver.ChromeOptions()
#     options.add_argument(f"user-agent={userAgent}")
    
#     # Инициализация драйвера с новыми опциями
#     driver = webdriver.Chrome(options=options)
    
#     try:
#         driver.get(f"https://realt.by/belarus/sale/flats/?page={i}")
#         driver.implicitly_wait(2)
#         elements = driver.find_elements(By.XPATH, value='//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div/div[1]/div[2]/div[1]/div/div')
#         for e in elements[0].find_elements(By.XPATH, '//a[contains(@aria-label, "Ссылка на объект")]'):
#             href = e.get_attribute('href')
#             with open('link.txt', 'a') as file:
#                 print(href, file=file)
#     except Exception as e:
#         print(f"Произошла ошибка: {e}")
#     finally:
#         driver.quit()  # Закрытие драйвера после каждой итерации
#     time.sleep(2)  # Пауза перед следующим запросом, чтобы снизить вероятность блокировки


# Парсим инфу

In [5]:
binary_components_path = "C:/Users/Asus/Desktop/DataScience/kaggle 6 + парсинг/chrome/chrome-win/chrome.exe"

driver_path = "C:/Users/Asus/Desktop/DataScience/kaggle 6 + парсинг/chrome/chromedriver_win32/chromedriver.exe"
ua = UserAgent()
userAgent = ua.random

options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={userAgent}") 
options.binary_location = binary_components_path
service = Service(driver_path)

driver = webdriver.Chrome(service=service, options=options)

data = []
ua = UserAgent()
counter = 0

print(f"Текущий userAgent: {userAgent}")
# links = ['https://realt.by/sale-flats/object/2806428/','https://realt.by/sale-flats/object/3162400/', 'https://realt.by/brest-region/sale-flats/object/3247042/', "https://realt.by/sale-flats/object/3170672/"]
with open('links.txt', 'r') as links:
    for link in links:
        # ua = UserAgent()
        # userAgent = ua.random

        # options = webdriver.ChromeOptions()
        # options.add_argument(f"user-agent={userAgent}") 
        # options.binary_location = binary_components_path
        # service = Service(driver_path)

        # driver = webdriver.Chrome(service=service, options=options)
        print(f'Link:{link}')
        counter+=1
        
        driver.get(link)
        try:
            check_price= WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[3]/div/div[1]/div[1]/div[1]'))).text
        except:
            try:
                driver.get(link)
                check_price= WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[3]/div/div[1]/div[1]/div[1]'))).text
            except:
                time.sleep(1)
                try:
                    driver.get(link)
                    check_price= WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[3]/div/div[1]/div[1]/div[1]'))).text
                except:
                    continue
        # Если есть цена
            
        if check_price != 'ДОГОВОРНАЯ ЦЕНА':
            record = {}  # Создаем пустой словарь для хранения данных
            #____________________________Записываем DATA and ID____________________________

            id_element = driver.find_elements(By.XPATH, '//*[@class="flex-shrink-0 md:pr-1 md:flex"]')[0].text
            id = id_element
            record['id'] = id
            #____________________________Записываем цену в долларах____________________________
            try:
                price_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'text-subhead sm:text-body text-basic inline-block')]")[2]
                price = ''.join(filter(str.isdigit, price_elements.text.strip()))
            except:
                try:
                    price = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[3]/div/div[1]/div[1]/div[1]/span').text
                except:
                    print('skip from price')
                    continue
            print(id, price)
            record['price'] = price

            #____________________________ Записываем основные параметры____________________________
            params_list_1 = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[4]/div[1]/div[2]/section[3]/ul')
            params_list_2 = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[3]/ul')
            params_list_3 = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[2]/ul')
            if len(params_list_2) == 1:
                params_list = params_list_2
            elif len(params_list_3) == 1:
                params_list = params_list_3
            else:
                params_list = params_list_1

            for element in params_list:
                # Получаем текст элемента и разделяем его на строки
                mas_param = element.text.split("\n")

                # Перебираем полученные строки парами
                for i in range(0, len(mas_param), 2):
                    # Проверяем, есть ли пара для текущего ключа, чтобы избежать выхода за пределы списка
                    if i + 1 < len(mas_param):  # Исправлено условие для корректной проверки
                        # Добавляем в словарь пару ключ-значение
                        record[mas_param[i]] = mas_param[i + 1]
            print('Запись основной инфы, success!)')
            #____________________________Записываем гео параметры____________________________

            params_list = driver.find_elements(By.XPATH, '//*[@id="map"]/ul')

            # Перебираем все найденные элементы 
            for params_container in params_list:
                # Для каждого контейнера ищем элементы li внутри него
                list_items = params_container.find_elements(By.CSS_SELECTOR, "li")

                # Перебираем все элементы li и извлекаем информацию
                for i in range(1,len(list_items)+1):
                    # Находим название параметра и его значение
                    param_name = driver.find_elements(By.XPATH, f'//*[@id="map"]/ul/li[{i}]/div/div[1]')[0].text
                    
                    param_value = driver.find_elements(By.XPATH, f'//*[@id="map"]/ul/li[{i}]/div/div[2]')[0].text
                    # Выводим название параметра и его значение
                    # print(f"i={i}, {param_name}: {param_value}")
                    record[param_name] = param_value  # Добавляем параметр и его значение в словарь

            print('Запись гео инфы, success!)')
            print(f'---------------------------------- it was link {counter}')
            data.append(record)

            with open('data.txt', 'a') as file:
                    print(record, file=file)
                        
driver.quit()  # Закрытие драйвера после каждой итерации

Текущий userAgent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Safari/605.1.15
Link:https://realt.by/brest-region/sale-flats/object/3242807/

ID 3242807 31800
Запись основной инфы, success!)
Запись гео инфы, success!)
---------------------------------- it was link 1
Link:https://realt.by/sale-flats/object/3242805/

ID 3242805 65000
Запись основной инфы, success!)
Запись гео инфы, success!)
---------------------------------- it was link 2
Link:https://realt.by/vitebsk-region/sale-flats/object/3242803/

ID 3242803 20300
Запись основной инфы, success!)
Запись гео инфы, success!)
---------------------------------- it was link 3
Link:https://realt.by/sale-flats/object/3242800/

ID 3242800 51000
Запись основной инфы, success!)
Запись гео инфы, success!)
---------------------------------- it was link 4
Link:https://realt.by/brest-region/sale-flats/object/3181220/

ID 3181220 22800
Запись основной инфы, success!)
Запись гео инфы, succes

In [ ]:
binary_components_path = "C:/Users/Asus/Desktop/DataScience/kaggle 6 + парсинг/chrome/chrome-win/chrome.exe"

driver_path = "C:/Users/Asus/Desktop/DataScience/kaggle 6 + парсинг/chrome/chromedriver_win32/chromedriver.exe"
ua = UserAgent()
userAgent = ua.random

options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={userAgent}") 
options.binary_location = binary_components_path
service = Service(driver_path)

driver = webdriver.Chrome(service=service, options=options)

link = 'https://realt.by/sale-flats/object/3261646/'
driver.get(link)
price_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'text-subhead sm:text-body text-basic inline-block')]")[2]
price = ''.join(filter(str.isdigit, price_elements.text.strip()))
print(price)
price_elements.text

27000


'≈ 27 000 $'

In [ ]:
import pandas as pd
import ast
pd.set_option('display.max_columns', None)
file_path = './chrome/data.txt'

data_dicts = []  # Список для хранения словарей

# Читаем файл построчно
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Преобразуем строку в словарь и добавляем в список
            data_dict = ast.literal_eval(line.strip())
            data_dicts.append(data_dict)
        except SyntaxError as e:
            print(f"Произошла ошибка синтаксиса: {e}")

# Преобразуем список словарей в DataFrame
df = pd.DataFrame(data_dicts)

df
# вчера это 27.02

,ID,Price,Количество комнат,Раздельных комнат,Площадь общая,Площадь жилая,Площадь кухни,Год постройки,Этаж / этажность,Тип дома,Планировка,Балкон,Новостройка,Ремонт,Стоянка автомобиля,Собственность,Условия продажи,Номер договора,Область,Населенный пункт,Улица,Район города,Микрорайон,Координаты,Санузел,Район,Номер дома,Площадь по СНБ,Возможен торг,Высота потолков,Площадь балконов,Полы,Мебель,Видео-домофон,Телефон,Огороженная территория,Гараж,Вид этажа,Сигнализация,Год капитального ремонта,Жилой комплекс,Варианты обмена,Сельсовет,Направление,Этаж,Число уровней,Аукцион
0,ID 2806428,≈ 33 700 $,1,1,30 м²,16 м²,7 м²,2027,11 / 15,Каркасно-блочный,Свободная планировка,Лоджия,Да,Без отделки,Есть,Частная,Чистая продажа,1/1 от 04.01.2022,Минская область,г. Минск,Мира просп.,Октябрьский район,Минск Мир (Minsk World),"53.8637, 27.542404",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID 3216141,≈ 79 000 $,3,3,61 м²,39.7 м²,8.2 м²,1986,9 / 9,Кирпичный,NaN,Лоджия застекленная,NaN,Евроремонт,NaN,Частная,Чистая продажа,23418,Брестская область,г. Брест,Машерова просп.,Ленинский район,Центр,"52.086084, 23.691745",Раздельный,Брестский район,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID 3264122,≈ 43 500 $,2,2,45.1 м²,26.5 м²,6.5 м²,1992,5 / 5,Кирпичный,NaN,Лоджия застекленная,NaN,NaN,NaN,Частная,Чистая продажа,24040,Брестская область,г. Брест,Дубровская ул.,Ленинский район,Дубровка,"52.133375, 23.650548",Раздельный,Брестский район,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID 2562548,≈ 92 874 $,2,2,67.3 м²,65.1 м²,NaN,NaN,4 / 10,Каркасно-блочный,NaN,Балкон,NaN,NaN,NaN,Частная,Чистая продажа,NaN,Минская область,г. Минск,Ильянская ул.,Центральный район,Лебяжий (Ржавец),"53.93437, 27.469058",NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID 3229926,≈ 46 500 $,3,3,49.2 м²,33.9 м²,5.3 м²,1973,5 / 5,Панельный,NaN,Балкон,NaN,NaN,NaN,Частная,Обмен,75/1 от 15.01.2024,Брестская область,г. Брест,Партизанский просп.,Московский район,Восток,"52.106411, 23.760331",NaN,Брестский район,NaN,51.6 м²,Да,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,ID 3266161,≈ 67 000 $,2,2,55.4 м²,35.9 м²,6.7 м²,1950,2 / 2,Кирпичный,Сталинка,Балкон,NaN,Удовлетворительный,NaN,Частная,Чистая продажа,NaN,Минская область,г. Минск,Коллекторная ул.,Московский район,"Немига, Короля, Клары Цеткин","53.9017, 27.541",Совмещенный,NaN,12,NaN,Да,3.02 м,0.5 м²,Деревянные,NaN,NaN,Есть,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885,ID 3219480,≈ 117 000 $,3,3,70 м²,60 м²,10 м²,2018,19 / 22,Каркасно-блочный,Свободная планировка,Лоджия,NaN,Отличный,NaN,Частная,Чистая продажа,NaN,Минская область,г. Минск,Братская ул.,Октябрьский район,Минск Мир (Minsk World),"53.8697, 27.5526",Раздельный,NaN,8,NaN,NaN,2.7 м,5 м²,Ламинированные,Есть,Есть,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886,ID 3259967,≈ 30 500 $,3,3,56 м²,39.3 м²,5.7 м²,1966,4 / 5,Панельный,Хрущевка,Балкон,NaN,NaN,NaN,Частная,Чистая продажа,NaN,Гродненская область,г. Лида,Крупской ул.,NaN,NaN,"53.894, 25.2741",NaN,Лидский район,163,NaN,NaN,NaN,NaN,Ламинированные,Есть,NaN,Есть,NaN,NaN,NaN,NaN,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
887,ID 3257423,≈ 6 500 $,1,1,28.8 м²,17.5 м²,5.8 м²,1979,2 / 9,Кирпичный,Малосемейка,NaN,NaN,Удовлетворительный,NaN,Частная,Чистая продажа,NaN,Гомельская область,г. Светлогорск,Свердлова ул.,NaN,NaN,"52.622594, 29.746435",NaN,Светлогорский район,27,NaN,Да,NaN,NaN,Деревянные,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Рабочий без дата и продавца

In [ ]:
# binary_components_path = "C:/Users/Asus/Desktop/DataScience/kaggle 6 + парсинг/chrome/chrome-win/chrome.exe"

# driver_path = "C:/Users/Asus/Desktop/DataScience/kaggle 6 + парсинг/chrome/chromedriver_win32/chromedriver.exe"

# options = webdriver.ChromeOptions() 
# options.binary_location = binary_components_path
# service = Service(driver_path)

# driver = webdriver.Chrome(service=service, options=options)

# data = []
# ua = UserAgent()
# counter = 0

# # with open('links.txt', 'r')as links:
# # links = ['https://realt.by/sale-flats/object/2806428/','https://realt.by/sale-flats/object/3162400/', 'https://realt.by/brest-region/sale-flats/object/3247042/', "https://realt.by/sale-flats/object/3170672/"]
# with open('links.txt', 'r') as links:
#     for link in links:
#         counter+=1

#         # Генерация нового userAgent
#         # userAgent = ua.random
#         # print(f"Текущий userAgent: {userAgent}")
        
#         # options = webdriver.ChromeOptions()
#         # options.add_argument(f"user-agent={userAgent}")
        
#         # # Инициализация драйвера с новыми опциями
#         # driver = webdriver.Chrome(options=options)
        
#         driver.get(link)

#         check_price= WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[3]/div/div[1]/div[1]/div[1]'))).text
#         # Если есть цена
#         if check_price != 'ДОГОВОРНАЯ ЦЕНА':
            
#             record = {}  # Создаем пустой словарь для хранения данных
#             #____________________________Записываем ID____________________________

#             element_id = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[1]/div/span/span[5]'))).text
#             record['ID'] = element_id

#             #____________________________Записываем цену в долларах____________________________
#             element_price= WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[3]/div/div[1]/div[1]/div[1]/span'))).text
#             record['Price'] = element_price
#             print(element_id, element_price)

#             #____________________________ Записываем основные параметры____________________________
#             params_list_1 = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[4]/div[1]/div[2]/section[3]/ul')
#             params_list_2 = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[3]/ul')
#             params_list_3 = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[2]/ul')
#             if len(params_list_2) == 1:
#                 params_list = params_list_2
#             elif len(params_list_3) == 1:
#                 params_list = params_list_3
#             else:
#                 params_list = params_list_1

#             # try:
#             #     element_price= WebDriverWait(driver, 10).until(
#             #     EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[3]/ul'))).text
#             #     params_list = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[3]/ul')
#             # except:
#             #     params_list = driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div[1]/div[2]/section[2]/ul')

#             for element in params_list:
#                 # Получаем текст элемента и разделяем его на строки
#                 mas_param = element.text.split("\n")

#                 # Перебираем полученные строки парами
#                 for i in range(0, len(mas_param), 2):
#                     # Проверяем, есть ли пара для текущего ключа, чтобы избежать выхода за пределы списка
#                     if i + 1 < len(mas_param):  # Исправлено условие для корректной проверки
#                         # Добавляем в словарь пару ключ-значение
#                         record[mas_param[i]] = mas_param[i + 1]
#             print('Запись основной инфы, success!)')
#             #____________________________Записываем гео параметры____________________________

#             params_list = driver.find_elements(By.XPATH, '//*[@id="map"]/ul')

#             # Перебираем все найденные элементы 
#             for params_container in params_list:
#                 # Для каждого контейнера ищем элементы li внутри него
#                 list_items = params_container.find_elements(By.CSS_SELECTOR, "li")

#                 # Перебираем все элементы li и извлекаем информацию
#                 for i in range(1,len(list_items)+1):
#                     # Находим название параметра и его значение
#                     param_name = driver.find_elements(By.XPATH, f'//*[@id="map"]/ul/li[{i}]/div/div[1]')[0].text
                    
#                     param_value = driver.find_elements(By.XPATH, f'//*[@id="map"]/ul/li[{i}]/div/div[2]')[0].text
#                     # Выводим название параметра и его значение
#                     # print(f"i={i}, {param_name}: {param_value}")
#                     record[param_name] = param_value  # Добавляем параметр и его значение в словарь

#             print('Запись гео инфы, success!)')
#             print(f'---------------------------------- it was link {counter}')
#             data.append(record)

#             with open('data.txt', 'a') as file:
#                     print(record, file=file)
                    
# driver.quit()  # Закрытие драйвера после каждой итерации

""
